<a href="https://www.kaggle.com/code/rubanzasilva/kaggle-s05e10?scriptVersionId=272405028" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Kaggle Playground Series Season 5 Episode 10

In this notebook, Iam going to use this to create a solution / predictive model for a kagge competition - Playground season 5 episode 10- whose link is https://www.kaggle.com/competitions/playground-series-s5e10

## About the competition

### Part 1

Goal :- Predict the likelihood of accidents on different types of roads.

Problem type :- Regression with target variable values ranging from 0 - 1

Metric/Evaluation :- Submissions are evaluated using the Root Mean Squared Error between the predicted and the observed target. 

I have previosly developed solutions to kaggle competitions specifically the playground competitions, below are examples you can use for reference

- https://www.kaggle.com/code/rubanzasilva/eda-fast-ai-neural-nets-gradient-boosting
- https://www.kaggle.com/code/rubanzasilva/s5e7-autogluon
- https://www.kaggle.com/code/rubanzasilva/s5e7-fastai-gradient-boosting
- https://www.kaggle.com/code/rubanzasilva/fastai-gradient-boosting

I expect to try out a number of algorithms not limited to 

- Decision Trees
- Gradient boosting methods like XGBoost, LightGBM, CatBoost, HistGradientBoosting 
- Neural Networks
- AutoML solutions suc as AutoGluon, h2o automl, TabFN, etc.
- Ensembles i.e. averages, voting, stacking and hill climbing

I will be heavily using the fastai library. 

I will use it to prepare my data for machine learning, by normalizing, categorizing, filling in missing values, putting variables into categorical or continous, the documentation for this is located at https://docs.fast.ai/tutorial.tabular.html.md. 

Once preprocessing is done, we can put it in a format thact enables us to use this data with external libraries like random forests, xgboost etc.

## Library Installation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
!uv pip install catboost
#%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
!uv pip install seaborn
!uv pip install xgboost
!uv pip install lightgbm
#%pip install fastkaggle
#%pip install h2o
#%pip install -Uqq fastbook
#%pip install polars
#%pip install -q -U autogluon.tabular
#%pip install autogluon
#%pip install --upgrade pip
!uv pip install tqdm
#%pip install wandb
#%pip install sweetviz
#%pip install xlearn

In [ ]:
!pip install -U scikit-learn

In [ ]:
import sklearn
print(sklearn.__version__)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from pathlib import Path

from fastai.tabular.all import *
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
from xgboost import plot_importance

import lightgbm as lgb
import catboost as cat

In [3]:
path = Path('/kaggle/input/playground-series-s5e10')
path

Path('/kaggle/input/playground-series-s5e10')

In [4]:
!ls /kaggle/input/playground-series-s5e10

sample_submission.csv  test.csv  train.csv


In [5]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

## Exploratory Data Analysis

In [ ]:
train_df

In [ ]:
train_df.head().T

In [ ]:
train_df.columns

In [ ]:
train_df.info()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.describe().T

In [ ]:
train_df.hist(figsize=(20,15),edgecolor='black');

- It seems like there a roads with 0 accident risk


In [ ]:
train_df.describe(include=[object]).T

In [ ]:
def plot_top_categories(data, column, top_n=20, figsize=(10,15)):
    # Get value counts and take top N
    top_values = data[column].value_counts().head(top_n)
    
    plt.figure(figsize=figsize)
    sns.barplot(x=top_values.values, y=top_values.index)
    plt.title(f'Top {top_n} {column} Categories')
    plt.xlabel('Count')
    plt.ylabel(column)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_top_categories(train_df, 'road_type', top_n=5)

In [ ]:
plot_top_categories(train_df, 'time_of_day', top_n=5)

In [ ]:
plot_top_categories(train_df, 'weather', top_n=5)

In [ ]:
plot_top_categories(train_df, 'lighting', top_n=5)

In [ ]:
def cross_val_predict(model, X_train, y_train, X_test, n_splits=5, random_state=42, 
                       return_proba=True, scoring='roc_auc', verbose=True):
    """
    Generic cross-validation function that works with ANY sklearn-compatible model.
    
    Parameters:
    - model: INSTANTIATED model object (e.g., xgb.XGBClassifier(n_estimators=100))
    - X_train, y_train: training data and labels
    - X_test: test data for final predictions
    - n_splits: number of CV folds
    - random_state: random seed for reproducibility
    - return_proba: if True, return probabilities; if False, return class predictions
    - scoring: metric to use ('roc_auc', 'accuracy')
    - verbose: print progress
    
    Returns:
    - oof_predictions: out-of-fold predictions on training set
    - test_predictions: predictions on test set (averaged across folds)
    - mean_score: mean score across folds
    - fold_scores: list of scores for each fold
    """
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import accuracy_score, roc_auc_score
    from sklearn.base import clone
    import numpy as np
    
    # Initialize stratified k-fold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Initialize arrays to store results
    if return_proba:
        oof_predictions = np.zeros(len(X_train))
        test_predictions = np.zeros(len(X_test))
    else:
        oof_predictions = np.zeros(len(X_train), dtype=int)
        test_predictions = np.zeros(len(X_test))
    
    fold_scores = []
    
    # Perform cross-validation
    for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
        if verbose:
            print(f"Training fold {fold}/{n_splits}...")
        
        # Split data for current fold
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Clone the model to avoid interference between folds
        model_fold = clone(model)
        
        # Train model on current fold
        model_fold.fit(X_fold_train, y_fold_train)
        
        # Get predictions based on return_proba setting
        if return_proba:
            # Get probabilities for positive class
            y_pred_proba = model_fold.predict_proba(X_fold_val)[:, 1]
            oof_predictions[val_index] = y_pred_proba
            test_predictions += model_fold.predict_proba(X_test)[:, 1] / n_splits
            
            # Calculate score
            if scoring == 'roc_auc':
                cv_score = roc_auc_score(y_fold_val, y_pred_proba)
            else:  # accuracy
                y_pred_fold = (y_pred_proba > 0.5).astype(int)
                cv_score = accuracy_score(y_fold_val, y_pred_fold)
        else:
            # Get class predictions
            y_pred_fold = model_fold.predict(X_fold_val)
            oof_predictions[val_index] = y_pred_fold
            test_predictions += model_fold.predict(X_test) / n_splits
            cv_score = accuracy_score(y_fold_val, y_pred_fold)
        
        fold_scores.append(cv_score)
        if verbose:
            print(f"Fold {fold} {scoring}: {cv_score:.6f}")
    
    # Round test predictions if using hard predictions
    if not return_proba:
        test_predictions = np.round(test_predictions).astype(int)
    
    # Calculate mean score
    mean_score = np.mean(fold_scores)
    if verbose:
        print(f"\nMean {scoring}: {mean_score:.6f}")
    
    return oof_predictions, test_predictions, mean_score, fold_scores


# =============================================================================
# USAGE EXAMPLES WITH DIFFERENT MODELS
# =============================================================================

# Example: CatBoost (if installed)
"""
cat_model = CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    random_state=42,
    verbose=False
)

cat_oof, cat_test, cat_auc, cat_scores = cross_val_predict(
    model=cat_model,
    X_train=X_train,
    y_train=y_train, 
    X_test=X_test,
    return_proba=True,
    scoring='roc_auc'
)
"""

# =============================================================================
# ENSEMBLE ALL MODELS
# =============================================================================
"""
print("Individual Model Performance:")
print(f"XGBoost AUC: {xgb_auc:.6f}")
print(f"Random Forest AUC: {rf_auc:.6f}")
print(f"LightGBM AUC: {lgb_auc:.6f}")

# Simple ensemble
ensemble_oof = (xgb_oof + rf_oof + lgb_oof) / 3
ensemble_test = (xgb_test + rf_test + lgb_test) / 3

ensemble_auc = roc_auc_score(y_train, ensemble_oof)
print(f"\nEnsemble AUC: {ensemble_auc:.6f}")

# Create submission
submission = pd.DataFrame({
    'id': test_df.index,
    'Depression': ensemble_test
})
submission.to_csv('ensemble_submission.csv', index=False)

print(f"\nShapes:")
print(f"OOF predictions: {ensemble_oof.shape} (training data)")
print(f"Test predictions: {ensemble_test.shape} (test data)")
print("Different shapes = different datasets!")
"""

### Prepare data for machine learning

Here I am going to take my data through some preprocessing steps. First I shall start by categorizing my variables into categorical and continous variables using fastai's `cont_cat_split` function.

In [6]:
#splits = RandomSplitter(valid_pct=0.2)(range_of(original_df))
#train_df = pd.concat([train_df, original_df], ignore_index=True) *

cont_names,cat_names = cont_cat_split(train_df, dep_var='accident_risk')

In [7]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [8]:
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='accident_risk',
                   y_block=RegressionBlock(),
                   splits=splits)

In [9]:
dls = to.dataloaders(bs=64)
#dls = to.dataloaders(bs=102


In [10]:
test_dl = dls.test_dl(test_df)
test_dl

In [11]:
test_dl.xs

,road_type,num_lanes,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,curvature
id,,,,,,,,,,,,
517754,1,2,3,3,1,2,2,1,2,2,2,-0.546220
517755,3,3,3,2,2,2,1,1,2,1,1,-1.647167
517756,3,2,2,2,1,2,1,1,2,2,2,0.371236
517757,2,4,2,1,3,1,1,1,1,1,3,1.692372
517758,1,2,2,1,1,2,1,2,1,2,4,1.362088
...,...,...,...,...,...,...,...,...,...,...,...,...
690334,2,2,3,2,3,1,1,1,2,2,3,-1.757262
690335,2,1,5,1,2,1,2,1,1,1,3,0.921710
690336,3,2,5,2,1,1,1,2,2,2,2,-1.280185


In [12]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

## Baseline model 

I will use something simple that works right out the box, then try to use more complex algorithms which can potentially improve the performance. I will start with decision trees specifically random forests but before that we shall start by doing some preprocessing with the fastai library.

In [ ]:
cat_names, len(cat_names)

In [ ]:
cont_names,len(cont_names)

### Random Forests

In [ ]:
#rf_regr = RandomForestRegressor(n_estimators= 100, max_depth=None, min_samples_split=2, min_samples_leaf=1, 
#                                max_features='sqrt', bootstrap=True, random_state=42, n_jobs=1)
rf_regr = RandomForestRegressor(100, min_samples_leaf=3)
rf_regr.fit(X_train,y_train)

Sklearn's random forest regressor has the below params

In [ ]:
rf_preds = rf_regr.predict(test_dl.xs)
rf_sc_preds = rf_regr.predict(X_test)

In [ ]:
rf_sc = root_mean_squared_error(y_test, rf_sc_preds)
rf_sc

In [ ]:
rf_preds,rf_preds.shape

### Feature Importance

In [ ]:
def rf_feat_importance(m, train_subset):
    return pd.DataFrame({'cols':train_subset.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
fi = rf_feat_importance(rf_regr, X_train)
#fi[:10]

fi

In [ ]:
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

#plot_fi(fi[:30]);
plot_fi(fi);

#### Submission

In [ ]:
sub_df

In [ ]:
!rm submission.csv
sub_df['accident_risk'] = rf_preds
sub_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv', index_col='id')
sub

## Gradient Boosting

### XGBoost

In [13]:
%%time
xgb_regr = xgb.XGBRegressor()
xgb_regr

CPU times: user 189 µs, sys: 0 ns, total: 189 µs
Wall time: 198 µs


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [14]:
xgb_regr = xgb_regr.fit(X_train, y_train)
xgb_regr

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [15]:
xgb_preds = xgb_regr.predict(test_dl.xs)
xgb_sc_preds = xgb_regr.predict(X_test)

In [16]:
xgb_sc = root_mean_squared_error(y_test, xgb_sc_preds)
xgb_sc

0.056337952613830566

In [17]:
xgb_preds,xgb_preds.shape

(array([0.29305762, 0.12260231, 0.18643875, ..., 0.2524025 , 0.12736891, 0.4863786 ], dtype=float32),
 (172585,))

In [19]:
with open('xgb_regr.pkl', 'wb') as f:
    pickle.dump(xgb_regr, f)

In [20]:
!ls

catboost_info  state.db  submission.csv  xgb_regr.pkl


### Feature Importance

In [ ]:
plot_importance(xgb_regr)

In [ ]:
!rm submission.csv
sub_df = pd.read_csv(path/'sample_submission.csv')
sub_df['accident_risk'] = xgb_preds
sub_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv', index_col='id')
sub

### LightGBM

In [ ]:
lgb.LGBMRegressor?

In [ ]:
lgbm_regr = lgb.LGBMRegressor()
lgbm_regr

In [ ]:
lgbm_regr = lgbm_regr.fit(X_train,y_train)
lgbm_regr

In [ ]:
lgbm_preds = lgbm_regr.predict(test_dl.xs)
lgbm_sc_preds = lgbm_regr.predict(X_test)

In [ ]:
lgbm_sc = root_mean_squared_error(y_test, lgbm_sc_preds)
lgbm_sc

In [ ]:
lgbm_preds,lgbm_preds.shape

In [ ]:
!rm submission.csv
sub_df = pd.read_csv(path/'sample_submission.csv')
sub_df['accident_risk'] = lgbm_preds
sub_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv', index_col='id')
sub

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Fix the TabularPandas setup for REGRESSION (remove CategoryBlock)
cont_names, cat_names = cont_cat_split(train_df, dep_var='accident_risk')

to = TabularPandas(train_df, 
                   procs=[Categorify, FillMissing, Normalize],
                   cat_names=cat_names,
                   cont_names=cont_names,
                   y_names='accident_risk',
                   # Remove y_block=CategoryBlock() for regression
                   # Or explicitly use: y_block=RegressionBlock()
                   splits=splits)

# Extract preprocessed data
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# LightGBM parameters for regression
params = {
    'objective': 'regression',  # For regression task
    'metric': 'rmse',           # Root Mean Squared Error
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1
}

# Train the model
print("Training LightGBM model...")
model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, valid_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluate the model
print("\n" + "="*50)
print("Model Evaluation Results")
print("="*50)
print("\nTraining Set:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.4f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.4f}")
print(f"R2 Score: {r2_score(y_train, y_pred_train):.4f}")

print("\nValidation Set:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.4f}")
print(f"R2 Score: {r2_score(y_test, y_pred_test):.4f}")

# Feature importance
print("\n" + "="*50)
print("Top 10 Most Important Features")
print("="*50)
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)
print(importance_df.head(10))

# Predict on test_df (the one without labels)
# First preprocess test_df using the same TabularPandas object
test_dl = dls.test_dl(test_df)
X_test_final = test_dl.xs
test_predictions = model.predict(X_test_final)

# Save predictions if needed
# test_df['predicted_accident_risk'] = test_predictions
# test_df.to_csv('predictions.csv', index=False)

In [ ]:
test_predictions.shape

In [ ]:
!rm submission.csv
sub_df = pd.read_csv(path/'sample_submission.csv')
sub_df['accident_risk'] = test_predictions
sub_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv', index_col='id')
sub

### CatBoost

In [ ]:
import catboost as cat

In [ ]:
cat.CatBoostRegressor??

In [ ]:
cat_regr = cat.CatBoostRegressor(loss_function='RMSE')
cat_regr

In [ ]:
cat_regr = cat_regr.fit(X_train, y_train)
cat_regr

In [ ]:
cat_preds = cat_regr.predict(test_dl.xs)
cat_sc_preds = cat_regr.predict(X_test)

In [ ]:
!rm submission.csv
sub_df = pd.read_csv(path/'sample_submission.csv')
sub_df['accident_risk'] = cat_preds
sub_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv', index_col='id')
sub

In [ ]:
def gbm_mdl_trng(model,metric_func, my_test_dl=test_dl, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
  model = model.fit(X_train, y_train)
  model_preds = tensor(model.predict(my_test_dl.xs))
  model_sc_preds = tensor(model.predict(X_test))
  model_score = metric_func(y_test,model_sc_preds)
  return model, model_score, model_preds, model_preds.shape

In [ ]:
cat_params = {'loss_function':'RMSE'}
cat_model = cat.CatBoostRegressor(**cat_params)
metric_func = root_mean_squared_error

In [ ]:
cat_model, cat_score, cat_preds, cat_shape = gbm_mdl_trng(cat_model,metric_func)
cat_score, cat_preds, cat_shape

In [ ]:
cat_preds

In [ ]:
!ls

In [ ]:
!rm submission.csv
sub_df = pd.read_csv(path/'sample_submission.csv')
sub_df['accident_risk'] = cat_preds
sub_df.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv', index_col='id')
sub